In [ ]:
import os

from dotenv import load_dotenv, dotenv_values 

from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

load_dotenv() 

# from llama_index.llms.bedrock import Bedrock

# llm = Bedrock(
#     model="us.meta.llama3-2-3b-instruct-v1:0",
#     region_name=os.getenv('AWS_REGION'),
#     aws_access_key_id=os.getenv('AWS_KEY_ID'),
#     aws_secret_access_key=os.getenv('AWS_SECRET_KEY')
# ) 
llm = Ollama(model="llama3.2:latest", request_timeout=120.0)

Settings.llm = llm

In [ ]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import StorageContext

neo4j_graph_store = Neo4jGraphStore(username=os.getenv('NEO4J_USERNAME'),password=os.getenv('NEO4J_PASSWORD'), url=os.getenv('NEO4J_URI'))
storage_context = StorageContext.from_defaults(graph_store=neo4j_graph_store)

print(os.getenv('NEO4J_URI'))

print(neo4j_graph_store.get_schema())

In [ ]:
from llama_index.core.prompts.base import (
    PromptTemplate,
    PromptType,
)
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

ENTITY_EXTRACT_TMPL_STR = """
A question is provided below. 
Given the question, extract up to {max_keywords} information that identify a given location in the question. Avoid stopwords.
Focus on extracting complete information from question, it can be more than one single word.
---------------------
{question}
---------------------
Provide information in the following comma-separated format: 'KEYWORDS: <information>'
"""

ENTITY_EXTRACT_PROMPT = PromptTemplate(
    ENTITY_EXTRACT_TMPL_STR,
    prompt_type=PromptType.QUERY_KEYWORD_EXTRACT,
)

AMAZON_NEPTUNE_NL2CYPHER_PROMPT_TMPL_STR = """
The query should be able to try best answer the question with the given graph schema.
The query should follow the following guidance:
- Fully qualify property references with the node's label.
```
// Incorrect
MATCH (p:SchoolZone)-[:HAS_SCHOOL]->(:School) RETURN p.name AS output
// Correct
MATCH (p:SchoolZone)-[:HAS_SCHOOL]->(i:School) RETURN i.name AS output
```
- Strictly follow the relationship on schema:
Given the relationship ['(:`School`)-[:`HAS_FLOOD_AREA`]->(:`FloodArea`)']:
```
// Incorrect
MATCH (a:FloodArea)<-[:HAS_FLOOD_AREA]-(t:School)
RETURN DISTINCT t.name AS output
// Correct
MATCH (a:School)-[:HAS_FLOOD_AREA]->(t:FloodArea)
RETURN DISTINCT t.name AS output
```
- Follow single direction (from left to right) query model:
```
// Incorrect
MATCH (a:School)<-[:HAS_FLOOD_AREA]-(t:FloodArea)
RETURN DISTINCT t.name AS output
// Correct
MATCH (a:School)-[:HAS_FLOOD_AREA]->(t:FloodArea)
RETURN DISTINCT t.name AS output
```
- The code criteria should always be on the source entity:
```
// Incorrect
MATCH (a:School)-[:HAS_FLOOD_AREA]->(t:FloodArea {code: 'A_CODE'})
RETURN DISTINCT t.name AS output
// Correct
MATCH (a:School {code: 'A_CODE'})-[:HAS_FLOOD_AREA]->(t:FloodArea)
RETURN DISTINCT t.name AS output
```
- The output variable must always be called 'output':
```
// Incorrect
MATCH (a:School)-[:HAS_FLOOD_AREA]->(t:FloodArea {code: 'A_CODE'})
RETURN DISTINCT t.name AS flood_area
// Correct
MATCH (a:School {code: 'A_CODE'})-[:HAS_FLOOD_AREA]->(t:FloodArea)
RETURN DISTINCT t.name AS output
```

Given any relationship property, you should just use them following the relationship paths provided, respecting the direction of the relationship path.
NOTE:
0. Try to get as much graph data as possible to answer the question
1. Put a limit of 30 results in the query.
---
Schema: 

Node properties are the following:
FloodArea {{name: STRING, code: STRING}},School {{name: STRING, code: STRING}},SchoolZone {{name: STRING, code: STRING}},Levee {{code: STRING, name: STRING}},Channel {{name: STRING, code: STRING}}
Relationship properties are the following:

The relationships are the following:
(:FloodArea)-[:PROTECTED_BY]->(:Levee),(:School)-[:HAS_FLOOD_AREA]->(:FloodArea),(:SchoolZone)-[:HAS_SCHOOL]->(:School),(:Levee)-[:PROTECTS_AGAINST]->(:Channel)
"""

NL2CYPHER_PROMPT = PromptTemplate(
    AMAZON_NEPTUNE_NL2CYPHER_PROMPT_TMPL_STR,
    prompt_type=PromptType.TEXT_TO_GRAPH_QUERY,    
)

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    entity_extract_template=ENTITY_EXTRACT_PROMPT,
    with_nl2graphquery=True,
    graph_query_synthesis_prompt=NL2CYPHER_PROMPT,
    graph_traversal_depth=3,
    verbose=True
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever,
    response_mode="refine"
)

In [ ]:
from IPython.display import display, Markdown

response = query_engine.query(
"""
what school zones are impacted by a flood of Channel with the 'LFZ_274' code?
""",
)

print(response)
